In [1]:
import sys
sys.path.append("/home/sara/lib")

In [2]:
import time
import random

import numpy
import libopf_py
import pandas as pd

from sklearn.svm import SVC
from sklearn import datasets
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score

In [3]:
#digits = datasets.load_digits()
#print("digits:   ---->>>> ", digits)

# To apply an classifier on this data, we need to flatten the image, to
# turn the data in a (samples, feature) matrix:

#n_samples = len(digits.images)
#data = digits.images.reshape((n_samples, -1))
#print("data: \n")
#print(data)
#print("n samples: \n")
#print(n_samples)
#print(len(data))


df = pd.read_csv("csv_dataset/dataframe.csv", header=None, sep=';')
df = df.drop([0], axis=0)
df = df.drop(columns=[0])
data = df.iloc[1:2975,1:137]
data = data.to_numpy()
data = data.astype('float64')
target = df.iloc[1:,136]
target = target.to_numpy()
target = target.astype('float64')
n_samples = len(data)

random.shuffle(target)
random.shuffle(target)

# Debugs
print("data: \n")
print(data)
print("n samples: \n")
print(n_samples)
print("target: ", target)
print(len(data))

data: 

[[218.  99. 268. ... 253. 410.   0.]
 [214.  87. 267. ... 246. 431.   0.]
 [216.  89. 267. ... 245. 422.   0.]
 ...
 [218.  84. 266. ... 244. 409.  22.]
 [213.  79. 261. ... 238. 402.  22.]
 [202.  75. 251. ... 233. 394.  22.]]
n samples: 

965
target:  [ 2.  9. 13. 21. 12. 13.  2.  0. 13. 21.  0. 21. 22. 22.  2.  0.  2. 13.
  2.  9. 21.  0. 13. 13. 21.  0.  0.  9. 12.  0.  2.  0.  9.  2. 13. 12.
  9.  9.  2. 13.  9. 21. 12. 12.  2. 12. 13.  0. 12.  2. 21.  2. 21. 12.
 21.  9. 22. 21.  0. 22.  2. 22. 21. 21. 13. 12. 12. 12. 12.  2. 22.  0.
  0.  0. 12.  2. 12. 22.  2.  9. 21. 13. 12. 22.  2. 13. 12. 21. 12. 21.
  0.  9.  9. 13. 21. 13.  9.  2.  9.  9.  2.  0. 21.  0.  9. 12. 12. 22.
 22.  2.  9. 21. 13. 13. 12.  9.  0. 12.  2.  9. 22. 13.  0.  2.  0.  0.
  2.  9. 12. 12. 12. 21.  2. 21.  0. 21. 12. 13.  9. 13.  0.  9.  9.  2.
  9.  9. 13. 12. 12.  9. 13.  9. 13.  9.  2.  2.  9. 21.  9.  9. 21. 22.
 13.  9. 13.  9.  2. 22. 12.  9. 22. 22. 13. 22. 12. 12.  9.  9. 12.  9.
  9. 22.

In [4]:
def run(split):
    n_split = int(split * n_samples)

    print("")
    print("=" * 100)
    print("")

    print("Split: %3.2f" % split)
    print("Size: %d, Classifying Size: %d, Testing Size: %d" % (n_samples, n_split, n_samples - n_split))

    rand = numpy.random.permutation(n_samples)

    #print(data)
    #print(target)
    
    random_data = data[rand]
    random_label = target[rand]

    data_train, data_test = random_data[:n_split], random_data[n_split:]
    label_train, label_test = random_label[:n_split], random_label[n_split:]

    print("-" * 20, "OPF", "-" * 20)

    def opf():
        
        # OPF only supports 32 bits labels at the moment
        label_train_32 = label_train.astype(numpy.int32)
        label_test_32 = label_test.astype(numpy.int32)

        O = libopf_py.OPF()

        t = time.time()
        O.fit(data_train, label_train_32)
        #    O.fit(data_train_32, label_train_32, learning="agglomerative", split=0.8)
        print("OPF: time elapsed in fitting: %f secs" % (time.time() - t))

        t = time.time()
        predicted = O.predict(data_test)
        print("predicted: \n", predicted)
        print("label_test: \n", label_test)
        print("OPF: time elapsed in predicting: %f secs" % (time.time() - t))

        print("Classification report for OPF:\n%s\n" % (classification_report(label_test_32, predicted)))
        print("Confusion matrix:\n%s" % confusion_matrix(label_test_32, predicted))
        
        acc_opf = accuracy_score(label_test_32, predicted)
        print("acuracia:", acc_opf)

    opf()

    print("-" * 20, "SVM", "-" * 20)

    def _svm():
        clf = SVC(gamma='auto')

        t = time.time()
        clf.fit(data_train, label_train)
        print("SVM: time elapsed in fitting: %f secs" % (time.time() - t))

        t = time.time()
        predicted = clf.predict(data_test)
        print("predicted: \n", predicted)
        print("label_test: \n", label_test)
        print("tamanhos do predicted e do label_test: ", len(predicted), len(label_test))
        print("SVM: time elapsed in predicting: %f secs" % (time.time() - t))

        print("Classification report for SVM:\n%s\n" % (classification_report(label_test, predicted)))
        print("Confusion matrix:\n%s" % confusion_matrix(label_test, predicted))
        
        # aqui eu vou calcular a acurácia.
        acc_svm = accuracy_score(label_test, predicted)
        print("acuracia:", acc_svm)

    _svm()

In [5]:
run(0.8)



Split: 0.80
Size: 965, Classifying Size: 772, Testing Size: 193
-------------------- OPF --------------------
OPF: time elapsed in fitting: 0.073668 secs
predicted: 
 [ 9 13 21 22  0  0  2 22 21 22 22 21 22  9  9  9 13  9  0 12 12 21 13 21
  0 22 12 12 12  2 13 13  0 12 13  0 12 12  2  0 22  9 22 13  0  2 12 21
 12  0 13 22 21 12 12  2  2  0 13 22 12  0  9 12 12 22 21 21 13 22 13 21
 13 21 21 12 22  0 21  2 12 21  2  0  0 12 12  9 12  9  9 21 21  0 21  9
 13  9  0 13  0  0 13 13 21  9 12 12 13 13 22 12  0 12 12 12  9 12 21  9
 12 21  2 12  0 21  0 12  9  0  0 21  9 12  9 13  0 22 21 21  2  2 21 22
 12  0 12 13 12 13 21  2  9 12  0  0  2 12 22  2  2 21 12  0 12 12 12  2
  2 13 21 12  0  0 21  9  2 13  9 21 12  9  2 12  0 22 12  0  2  2 12 13
 12]
label_test: 
 [12.  2. 13. 12. 21.  9. 12. 21.  2. 21.  9.  0.  2. 21.  2.  9. 21.  9.
 21.  9.  2.  2. 12.  9. 13.  2.  2.  2. 12. 22.  0. 13.  0.  9. 12.  0.
 13. 22.  9.  9. 21. 13.  2.  9.  9. 12.  9.  9. 21. 13. 13. 12.  9. 12.
 13.  2. 

/home/sara/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
